In [14]:
%pip install e2b==0.9.3
%pip install langchain
%pip install openai


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from e2b import DataAnalysis
import os
from IPython.display import Image

os.environ["E2B_API_KEY"] = "<E2B_API_KEY>"
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

# E2B Sandbox
session = DataAnalysis()

# Upload CSV file with data to the E2B sandbox
csv_file_remote_path: str
with open("/Users/mlejva/Developer/e2b-cookbook/guides/langchain_data_analysis/netflix.csv", "rb") as f:    
    csv_file_remote_path = session.upload_file(file=f)




In [16]:
import ast
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

class RunPythonCodeInput(BaseModel):
    code: str = Field(description="Python code to run", )

class RunPythonCodeTool(BaseTool):
    name = "run_python_code"
    description = "Run Python code from file in cloud sandbox. ALWAYS PRINT VARIABLES TO SHOW THE VALUE. You have access to the internet and can load and save files from the '/home/user' directory."
    
    args_schema: Type[BaseModel] = RunPythonCodeInput

    def _add_last_line_print(self, tree):
        node = tree.body[-1]
        if isinstance(node, ast.Expr) and isinstance(node.value, ast.Call):
            if isinstance(node.value.func, ast.Name) and node.value.func.id == 'print':
                return tree
        tree.body[-1] = ast.Expr(value=ast.Call(func=ast.Name(id='print', ctx=ast.Load()),
                                        args=[node.value],
                                        keywords=[]))
        return tree

    def _run(self, code: str):
        tree = ast.parse(code)
        tree = self._add_last_line_print(tree)
        code = ast.unparse(tree)
        stdout, stderr, artifacts = session.run_python(code)

        for artifact in artifacts:
            b = artifact.download()
            Image(data=b)

        return stdout, stderr
    
    def _arun(self, code: str):
        raise NotImplementedError("run_python_code does not support async")


In [17]:
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

llm = ChatOpenAI(model="gpt-4", temperature=0)
tools = [RunPythonCodeTool()]
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [18]:
agent.run(f"I uploaded a CSV file with data here '{csv_file_remote_path}', please load it and show me the first 5 rows.")

session.close()



> Entering new AgentExecutor chain...

Invoking: `run_python_code` with `{'code': "import pandas as pd\n\n# Load the CSV file\nnetflix_data = pd.read_csv('/home/user/netflix.csv')\n\n# Show the first 5 rows\nnetflix_data.head()"}`


CODE 1 import pandas as pd

# Load the CSV file
netflix_data = pd.read_csv('/home/user/netflix.csv')

# Show the first 5 rows
netflix_data.head()
CODE 2 import pandas as pd
netflix_data = pd.read_csv('/home/user/netflix.csv')
print(netflix_data.head())
('  show_id  ...                                        description\n0      s1  ...  As her father nears the end of his life, filmm...\n1      s2  ...  After crossing paths at a party, a Cape Town t...\n2      s3  ...  To protect his family from a powerful drug lor...\n3      s4  ...  Feuds, flirtations and toilet talk go down amo...\n4      s5  ...  In a city of coaching centers known to train I...\n\n[5 rows x 12 columns]', '')Here are the first 5 rows of your Netflix data:

|   | show_id | ... | descript